In [1]:
contest_folder = 'practice'

In [2]:
from analysis import get_current_status, select_solution

In [4]:
df_grouped, aggregated_df = get_current_status(contest_folder)

In [5]:
aggregated_list = aggregated_df.to_dict(orient='records')

In [6]:
aggregated_df

,problem_code,problem_name,statement,sample_in,sample_out,full_in,solution_id,execution_response,execution_code,execution_sample_out,execution_full_out,hash,status,timestring
0,line_by_line,Line by Line,You’ve found a solution to an implementation-h...,4\n2 50\n3 10\n13 37\n950 95\n,Case #1: 20.710678118654748\nCase #2: 11.54434...,174\n2 50\n3 10\n13 37\n950 95\n2 1\n2 2\n2 3\...,"[000, 002, 003]",[**Key Findings:**\n\n1. **Understanding Succe...,"[import math\n\ndef compute_increase(N, P):\n ...",[Case #1: 20.710678118654755\nCase #2: 11.5443...,[Case #1: 20.710678118654755\nCase #2: 11.5443...,50b1cb3a,not_analyzed,3407
1,line_of_delivery_part_1,Line of Delivery (Part 1),*This problem shares some similarities with pr...,4\n2 5\n7\n2\n3 1\n9\n5\n7\n4 7\n8\n7\n9\n6\n2...,Case #1: 1 2\nCase #2: 3 4\nCase #3: 3 0\nCase...,111\n2 5\n7\n2\n3 1\n9\n5\n7\n4 7\n8\n7\n9\n6\...,"[000, 001]",[**Key Findings:**\n\n1. **Unique Energy Const...,[import sys\nimport threading\n\ndef main():\n...,[Case #1: 1 2\nCase #2: 2 4\nCase #3: 2 0\nCas...,[Case #1: 1 2\nCase #2: 2 4\nCase #3: 2 0\nCas...,b6413a1a,not_analyzed,3407
2,line_of_delivery_part_2,Line of Delivery (Part 2),*This problem shares some similarities with pr...,5\n2 5\n7\n2\n3 6\n9\n5\n7\n4 8\n8\n7\n9\n6\n2...,Case #1: 1 2\nCase #2: 2 2\nCase #3: 2 0\nCase...,112\n2 5\n7\n2\n3 6\n9\n5\n7\n4 8\n8\n7\n9\n6\...,[003],[**Key Findings:**\n\n1. **Energy Transfer Mec...,[import sys\nimport threading\nimport bisect\n...,[Case #1: 1 2\nCase #2: 2 2\nCase #3: 1 1\nCas...,[Case #1: 1 2\nCase #2: 2 2\nCase #3: 1 1\nCas...,ce192dec,not_analyzed,3407


In [7]:
import time
import threading

while True:
    _, aggregated_df = get_current_status(contest_folder)
    if len(aggregated_df) == 0:
        print("Not outputs found yet")
        time.sleep(30)
        continue
    
    aggregated_list = aggregated_df.to_dict(orient='records')
    print(f"Running analysis at {aggregated_df['timestring'][0]}")
    
    for row in aggregated_list:
        if row["status"] != "not_analyzed":
            continue
        analysis_thread = threading.Thread(target=select_solution, args=(row,))
        analysis_thread.start()
        print(f"Running analysis at {row['timestring']} for {row['problem_name']}")

    time.sleep(30)

Running analysis at 3409
Running analysis at 3409 for Line by Line
Running analysis at 3409 for Line of Delivery (Part 1)
Running analysis at 3409 for Line of Delivery (Part 2)
Running analysis at 3439
Running analysis at 3439 for Fall in Line
Running analysis at 3439 for Line by Line
Running analysis at 3439 for Line of Delivery (Part 1)
Running analysis at 3439 for Line of Delivery (Part 2)
Running analysis at 3439 for Walk the Line
Running analysis at 3510
Running analysis at 3510 for Fall in Line
Running analysis at 3510 for Line of Delivery (Part 2)
Running analysis at 3510 for Walk the Line
Running analysis at 3541
Running analysis at 3541 for Line of Delivery (Part 1)
Running analysis at 3612
Running analysis at 3612 for Line of Delivery (Part 1)
Running analysis at 3612 for Line of Delivery (Part 2)
Running analysis at 3612 for Walk the Line
Running analysis at 3642
Running analysis at 3713
Running analysis at 3713 for Line of Delivery (Part 2)
Running analysis at 3744
Running 

KeyboardInterrupt: 

In [ ]:
process_row(aggregated_list[3])